
---

### This 📘 notebook needs to be run from VS Code with [these prerequisites](https://github.com/johnmaeda/SK-Recipes#-prerequisites-to-getting-started-with-the-notebooks-in-this-repo).

How to use this notebook: 

* Just read the text and scroll along until you run into code blocks.
* Code blocks have computer code inside them — hover over the block and you can run the code.
* Run the code by hitting the ▶️ "play" button to the left. If the code runs you'll see a ✔️. If not, you'll get a ❌.
* The output and status of the code block will appear just below itself — you need to scroll down further to see it.
* Sometimes a code block will ask you for input in a hard-to-notice dialog box 👆 at the top of your notebook window. 

---

# S1E7 • 🧂🍋🔥 Stay Fresh With APIs

## Step 1: Instantiate a 🔥 kernel 

In [ ]:
#r "nuget: Microsoft.SemanticKernel, *-*"
#r "nuget: Microsoft.SemanticKernel.Skills.Web, *-*"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.KernelExtensions;
using System.IO;
using Microsoft.SemanticKernel.Configuration;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.Orchestration;

IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder.Build();

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    kernel.Config.AddAzureOpenAICompletionBackend("davinci", model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAICompletionBackend("davinci", model, apiKey, orgId);

## Step 2: Access fresh data with the Bing Web Search API 🍋 connector

To get a Bing Web Search API key visit [this resource](https://www.microsoft.com/en-us/bing/apis/bing-web-search-api) to start. Once you have that in hand, you can now grab live search information to get added to the context of your choice. For example, we can find out what's the tallest building in Europe:

In [ ]:
using Microsoft.SemanticKernel.Skills.Web.Bing;

var bingApiKey = await InteractiveKernel.GetPasswordAsync("Please enter your Bing Search API key");
var bingConnector = new BingConnector(bingApiKey);

var webSearchEngineSkill = new WebSearchEngineSkill(bingConnector);
var web = kernel.ImportSkill(webSearchEngineSkill);

var bingResult = await kernel.RunAsync(
    "What's the tallest building in Europe?", 
    web["SearchAsync"]
);
Console.WriteLine(bingResult);

## Step 3: Combine existing 🥑 memories with fresh 🍋 data

So we take our memories to the next level by providing it "live" memories that are delivered via a connector. It's a terrific combination of timeless information and timely information.

In [ ]:
var myFunction = kernel.CreateSemanticFunction(@"
These are facts about me:
1/ I'm tall-ish: about 5ft 11in
2/ I like sushi — but not the kind with uni
3/ I've been to {{$input}}
Here is a short story under 100 words about me:
", maxTokens: 200, temperature: 0.1, topP: .1);

var result = await kernel.RunAsync(
    "The most beautiful place in Turkey?", 
    web["SearchAsync"],
    myFunction
);

Console.WriteLine(result);

# ⏭️ Next Steps

Run through more advanced examples in the notebooks that are available in our GitHub repo at [https://aka.ms/sk/repo](https://aka.ms/sk/repo).

Or stay a longer while and ask Bing to added new information from the Internet to fortify your LLM AI magic.